<a href="https://colab.research.google.com/github/paulinaeb/IDaSec-project/blob/exp3/my_exp/before_evasion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy --force-reinstall

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.2.0 which is incompatible.


In [2]:
!git clone --branch main https://github.com/paulinaeb/IDaSec-project.git

fatal: destination path 'IDaSec-project' already exists and is not an empty directory.


Libraries

In [3]:
!pip uninstall -y torch torchtext


Found existing installation: torch 2.2.0
Uninstalling torch-2.2.0:
  Successfully uninstalled torch-2.2.0
Found existing installation: torchtext 0.17.0
Uninstalling torchtext-0.17.0:
  Successfully uninstalled torchtext-0.17.0


In [4]:
# !pip install --upgrade torchtext
# !pip install --upgrade torch
!pip install torch==2.2.0 torchtext==0.17.0




  Using cached torch-2.2.0-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached torchtext-0.17.0-cp311-cp311-manylinux1_x86_64.whl.metadata (7.6 kB)
Using cached torch-2.2.0-cp311-cp311-manylinux1_x86_64.whl (755.5 MB)
Using cached torchtext-0.17.0-cp311-cp311-manylinux1_x86_64.whl (2.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.2.0 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.2.0 which is incompatible.


In [5]:
import os
import pandas as pd
import torch
import numpy
# nlp library of Pytorch
from torchtext import data

import warnings as wrn
wrn.filterwarnings('ignore')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [6]:
# Path to the dataset folder
dataset_path = "IDaSec-project/dataset"

# List all subfolders in dataset
subfolders = [f.name for f in os.scandir(dataset_path) if f.is_dir()]
print("available datasets:", subfolders)


available datasets: ['sms', 'enron1', 'enron2']


In [7]:
enron1_path = dataset_path + "/enron1"
enron2_path = dataset_path + "/enron2"
sms_path = dataset_path + "/sms"

In [8]:
enron1_files = os.listdir(enron1_path)
print(f"\nFiles in '{enron1_path}': {enron1_files}")

enron2_files = os.listdir(enron2_path)
print(f"\nFiles in '{enron2_path}': {enron2_files}")

sms_files = os.listdir(sms_path)
print(f"\nFiles in '{sms_path}': {sms_files}")


Files in 'IDaSec-project/dataset/enron1': ['enron1_val.csv', 'enron1_test.csv', 'enron1_train.csv']

Files in 'IDaSec-project/dataset/enron2': ['enron2_val.csv', 'enron2_train.csv', 'enron2_test.csv']

Files in 'IDaSec-project/dataset/sms': ['train.csv', 'val.csv', 'test.csv']


Enron1

In [9]:
enron1_train = pd.read_csv(enron1_path + "/enron1_train.csv")
enron1_test = pd.read_csv(enron1_path + "/enron1_test.csv")
enron1_val = pd.read_csv(enron1_path + "/enron1_val.csv")

In [10]:
enron1_train.head()

,email,target
0,"Subject: prom dress shopping hi , just wanted ...",ham
1,"Subject: hi agaain hello , welcome to pharm la...",spam
2,Subject: feedback monitor error - meter 984132...,ham
3,Subject: instructions to remove spyware / adwa...,spam
4,Subject: acrobat pro 7 . 0 $ 69 . 95 xp pro op...,spam


The model

In [11]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cuda.deterministic = True

In [12]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence


In [13]:
tokenizer = get_tokenizer("spacy", language="en_core_web_sm")

# Example: assuming your dataset has a 'text' column and a 'type' label (spam/ham)
def yield_tokens(data_iter):
  for index, row in data_iter.iterrows():
        yield tokenizer(row['email'])


In [14]:
enron1_train.head()

,email,target
0,"Subject: prom dress shopping hi , just wanted ...",ham
1,"Subject: hi agaain hello , welcome to pharm la...",spam
2,Subject: feedback monitor error - meter 984132...,ham
3,Subject: instructions to remove spyware / adwa...,spam
4,Subject: acrobat pro 7 . 0 $ 69 . 95 xp pro op...,spam


In [15]:
vocab = build_vocab_from_iterator(yield_tokens(enron1_train), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

# Optionally map labels (e.g., ham → 0, spam → 1)
label_map = {"ham": 0, "spam": 1}


In [16]:
class TextDataset(Dataset):
    def __init__(self, data, vocab, tokenizer, label_map):
        self.data = data
        self.vocab = vocab
        self.tokenizer = tokenizer
        self.label_map = label_map

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]  # <-- change this line
        text = torch.tensor(self.vocab(self.tokenizer(item['email'])), dtype=torch.long)
        label = torch.tensor(self.label_map[item['target']], dtype=torch.float)
        return text, label


In [17]:
def collate_batch(batch):
    texts, labels = zip(*batch)
    lengths = torch.tensor([len(t) for t in texts])
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=vocab["<pad>"])
    labels = torch.stack(labels)
    return padded_texts, lengths, labels


In [18]:
train_dataset = TextDataset(enron1_train, vocab, tokenizer, label_map)
val_dataset = TextDataset(enron1_val, vocab, tokenizer, label_map)
test_dataset = TextDataset(enron1_test, vocab, tokenizer, label_map)

BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)


In [19]:
import torch.nn as nn

class LSTMNet(nn.Module):

    def __init__(self,vocab_size,embedding_dim,hidden_dim,output_dim,n_layers,bidirectional,dropout):

        super(LSTMNet,self).__init__()

        # Embedding layer converts integer sequences to vector sequences
        self.embedding = nn.Embedding(vocab_size,embedding_dim)

        # LSTM layer process the vector sequences
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers = n_layers,
                            bidirectional = bidirectional,
                            dropout = dropout,
                            batch_first = True
                           )

        # Dense layer to predict
        self.fc = nn.Linear(hidden_dim * 2,output_dim)
        # Prediction activation function
        self.sigmoid = nn.Sigmoid()


    def forward(self,text,text_lengths):
        embedded = self.embedding(text)
        # Thanks to packing, LSTM don't see padding tokens
        # and this makes our model better
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths.cpu(), batch_first=True, enforce_sorted=False
        )

        packed_output,(hidden_state,cell_state) = self.lstm(packed_embedded)

        # Concatenating the final forward and backward hidden states
        hidden = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)

        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.sigmoid(dense_outputs)

        return outputs



In [20]:
# Replacing TEXT.vocab with our new vocab
SIZE_OF_VOCAB = len(vocab)
EMBEDDING_DIM = 100
NUM_HIDDEN_NODES = 64
NUM_OUTPUT_NODES = 1
NUM_LAYERS = 2
BIDIRECTION = True
DROPOUT = 0.2

model = LSTMNet(SIZE_OF_VOCAB,
                EMBEDDING_DIM,
                NUM_HIDDEN_NODES,
                NUM_OUTPUT_NODES,
                NUM_LAYERS,
                BIDIRECTION,
                DROPOUT
               )


In [21]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Binary Cross Entropy (for binary classification)
criterion = nn.BCELoss()
criterion = criterion.to(device)


In [22]:
def binary_accuracy(preds, y):
    # round predictions to 0 or 1
    rounded_preds = torch.round(preds)
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc


In [23]:
def train(model, dataloader, optimizer, criterion):
    epoch_loss = 0.0
    epoch_acc = 0.0

    model.train()

    for text, text_lengths, labels in dataloader:
        text = text.to(device)
        text_lengths = text_lengths.to(device)
        labels = labels.to(device).float()

        optimizer.zero_grad()

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, labels)
        acc = binary_accuracy(predictions, labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)


In [24]:
def evaluate(model, dataloader, criterion):
    epoch_loss = 0.0
    epoch_acc = 0.0

    model.eval()

    with torch.no_grad():
        for text, text_lengths, labels in dataloader:
            text = text.to(device)
            text_lengths = text_lengths.to(device)
            labels = labels.to(device).float()

            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, labels)
            acc = binary_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)


In [25]:
EPOCH_NUMBER = 15

for epoch in range(1, EPOCH_NUMBER + 1):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_loader, criterion)

    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print()


	Train Loss: 0.652 | Train Acc: 70.57%
	 Val. Loss: 0.620 |  Val. Acc: 71.49%

	Train Loss: 0.592 | Train Acc: 70.72%
	 Val. Loss: 0.574 |  Val. Acc: 71.49%

	Train Loss: 0.552 | Train Acc: 70.72%
	 Val. Loss: 0.542 |  Val. Acc: 71.49%

	Train Loss: 0.497 | Train Acc: 70.78%
	 Val. Loss: 0.477 |  Val. Acc: 71.61%

	Train Loss: 0.399 | Train Acc: 76.89%
	 Val. Loss: 0.355 |  Val. Acc: 82.19%

	Train Loss: 0.302 | Train Acc: 88.51%
	 Val. Loss: 0.287 |  Val. Acc: 89.05%

	Train Loss: 0.238 | Train Acc: 92.15%
	 Val. Loss: 0.239 |  Val. Acc: 90.96%

	Train Loss: 0.198 | Train Acc: 93.05%
	 Val. Loss: 0.208 |  Val. Acc: 92.16%

	Train Loss: 0.171 | Train Acc: 94.03%
	 Val. Loss: 0.194 |  Val. Acc: 92.16%

	Train Loss: 0.144 | Train Acc: 95.31%
	 Val. Loss: 0.176 |  Val. Acc: 93.61%

	Train Loss: 0.126 | Train Acc: 95.78%
	 Val. Loss: 0.167 |  Val. Acc: 93.85%

	Train Loss: 0.116 | Train Acc: 96.31%
	 Val. Loss: 0.152 |  Val. Acc: 94.34%

	Train Loss: 0.098 | Train Acc: 97.00%
	 Val. Loss: 

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def evaluate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for text, lengths, labels in data_loader:
            text, lengths, labels = text.to(device), lengths.to(device), labels.to(device)
            outputs = model(text, lengths).squeeze()

            preds = torch.round(outputs)

            all_preds.extend(preds.tolist())   # Use .tolist() instead of .numpy()
            all_labels.extend(labels.tolist())

    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    report = classification_report(all_labels, all_preds, zero_division=0)

    return acc, precision, recall, f1, report


In [31]:
acc, prec, rec, f1, report = evaluate_metrics(model, test_loader)
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification Report:\n", report)

Accuracy: 0.9489
Precision: 0.9033
Recall: 0.9249
F1-score: 0.9140

Classification Report:
               precision    recall  f1-score   support

         0.0       0.97      0.96      0.96       706
         1.0       0.90      0.92      0.91       293

    accuracy                           0.95       999
   macro avg       0.94      0.94      0.94       999
weighted avg       0.95      0.95      0.95       999

